# Introdution

Основной целью этого проекта является выбор модели для прогноза стоимости настольных игр.

Основной метрикой, на которую опирается проект выбрана МАРЕ, так как она наиболее наглядно показывает среднюю погрешность в процентах при определении стоимости игр. Вторичными метриками выбраны МАЕ и RMSE для фиксирования погрешностей при предсказаниях.

Задачи, которые необходимо выполнить для достижения цели:
1. Парсинг данных для обучения
2. Очистка данных
3. Feature engeneering
4. Исследование моделей
5. Выбор модели для решения поставленной задачи

Необходимо сразу обратить внимание, что для обучения был проведен парсинг двух дата-сетов:**книги** и **настольные игры**
Дата-сет с книгами был выбран для того, чтобы проверить как себя будут вести модели, если данных будет в несколько раз больше, чем в "настольных играх".

С каждым из дата-сетов будут проведены отдельные исследования. В дальнейшем есть идея использовать книги для предварительного обучения.



# Import

In [1]:
!pip install tensorflow == 2.3
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install pytorch-tabnet

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import seaborn as sns
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import pymorphy2
import math
from nltk.corpus import stopwords

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# pytorch
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric
# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers.schedules import *
import albumentations as a

# plt
import matplotlib.pyplot as plt
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Метрики МАЕ и МАРЕ

Метрика MAPE

In [3]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Метрика MAE:

In [4]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred-y_true))

Метрика rmse ( нужна для baseline'a):

In [5]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

Фиксируем рандом сид

In [6]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Подгружаем дата-сеты для обучения: Книги и настольные игры:

In [7]:
DATA_DIR = '../input/books-for-pretrain/'
data = pd.read_csv(DATA_DIR + 'books_df.csv')

In [8]:
DATA_DIR1 = '../input/tabletop-games/'
data_games = pd.read_csv(DATA_DIR1 + 'games_hg.csv')

Теперь приступим к изучению дата-сетов и моделей.

# Первый дата-сет (книги)

Первый дата-сет содержит в себе следующую информацию:

In [9]:
data.head()

Как видно, данные засорены различным мусором. Проведем предварительную очистку и обработку, необходимые для дальнейшей работы с дата-сетом.

# Очистка и обработка данных

Удалим мусор из ячеек:

In [10]:
data['description'] = data['description'].apply(lambda x: x.replace('\r', ''))
data['Name'] = data['Name'].apply(lambda x: x.replace('\r', ''))
data['description'] = data['description'].apply(lambda x: x.replace('\n', ''))
data['Name'] = data['Name'].apply(lambda x: x.replace('\n', ''))
data['description'] = data['description'].apply(
    lambda x: x.replace('Читать дальше…', ''))

Результат:

In [11]:
data.head()

# Небольшой Feature engeneering

Выделим несколько фич:
* Символьная длина описания
* Символьная длина названия

In [12]:
data['desc_length'] = data['description'].apply(lambda x: len(x))

In [13]:
data['name_length'] = data['Name'].apply(lambda x: len(x))

Результат:

In [14]:
data.head()

Посмотрим количество авторов и издательств:

In [16]:
data.author.value_counts()

In [17]:
data.more_info.value_counts()

Как видно, издательств и авторов очень много. В дальнейшем они будут отнесены к категориальным признакам.

Проверим количество строк:

In [15]:
len(data)

Проверим пропуски:

In [16]:
data.isna().sum()

Удалим мусор из target'a и переведем значения столбца в int:

In [17]:
data['price'] = data['price'].apply(lambda x: x.replace('\xa0', ''))

In [18]:
data['price'] = data['price'].apply(lambda x: int(x))

Предобработка данных:

In [19]:
def preproc_data(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'Name'], axis=1, inplace=True)

    scaler = StandardScaler()
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features, dummy_na=False)

    return df_output

Используемые для обучения столбцы:

In [20]:
num_features = ['name_length', 'desc_length']

In [21]:
cat_features = ['author', 'more_info']

Результат:

In [22]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

# Train-test split

Обработка и разделение данных на обучающую и валидационную выборки:

In [23]:
y = df_preproc.price.values     # наш таргет
X = df_preproc.drop(['price'], axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True, random_state=RANDOM_SEED)

# Примитивный бейзлайн

Обычный бейзлайн, по-большей мере, необходим для того, чтобы понимать, в какую сторону движется исследование моделей для прогнозирования стоимости, то есть является начальной точкой.

In [25]:
books_base_mape = (mape(y_test, np.mean(y_train)))*100
books_base_mae = mae(y_test, np.mean(y_train))
books_base_rmse = rmse(y_test, np.mean(y_train))

In [26]:
print(f"TEST mape: {books_base_mape:0.2f}%")

In [27]:
print(f"TEST mae: {books_base_mae:0.2f}")

In [28]:
print(f"TEST mae: {books_base_rmse:0.2f}")

 В данном случае метрика МАРЕ выходит больше 200%. Это означает, что в среднем, при прогнозе модель ошибается более чем в два раза, что является плохим результатом при прогнозировании стоимости книги.

# Линейная регрессия

Первым и наиболее простым решением является использование линейной регресси для предсказания стоимости. Ниже представлена реализация линейной регрессии:

In [29]:
linreg = LinearRegression().fit(X_train, np.log(y_train.reshape(-1, 1)))
test_predict = linreg.predict(X_test)

Проверим, есть ли выбросы:

In [30]:
plt.hist(test_predict)

Как можно увидеть из графика,есть небольшое количество значений разительно отличающееся от основной части значений. Посмотрим максимальное значение:

In [31]:
test_predict.max()

Уберем выбросы и посчитаем отслеживаемые метрики:

In [32]:
# Списки для хранения новых данных
y_test_new = []
pred = []

Убираем inf из массива с предиктами, и соответствующие им значения из y_test:

In [33]:
for i in range(0, len(y_test)-1):
    if np.exp(test_predict[i]) != np.inf:
        y_test_new.append(y_test[i])
        pred.append(np.exp(test_predict[i]))

Преобразуем новые списки в массивы для вычисления метрик:

In [34]:
y_test_new1 = np.array(y_test_new)
pred1 = np.array(pred)

Вычислим исследуемые метрики:

In [35]:
books_linreg_mape = (mape(y_test_new1, pred1))*100
books_linreg_mae = mae(y_test_new1, pred1)
books_linreg_rmse = math.sqrt(mean_squared_error(y_test_new1, pred1))

In [36]:
print(f"TEST mape: {books_linreg_mape:0.2f}%")

In [37]:
print(f"TEST mae: {books_linreg_mae:0.2f}")

In [38]:
print(f"TEST rmse: {books_linreg_rmse:0.2f}")

Погрешность существенно уменьшилась, но возникли непредвиденные трудности при работе с выбросами, появившимися после предсказания линейной регрессии. Важно отметить, что при других алгоритмов схожих трудностей не возникало.

# RandomForest

Обратимся к решающим деревьям. В качестве следующей модели используется RandomForestRegressor. Изначально была идея использовать с подбором гиперпараметров, но вычисления получаются безумно долгие. Поэтому параметры выбраны исходя из нескольких экспериментов и опыта предыдущих проектов.

In [39]:
rand_forest = RandomForestRegressor(n_estimators=500,
                                    n_jobs=-1,
                                    max_depth=15,
                                    max_features='log2',
                                    random_state=RANDOM_SEED,
                                    oob_score=True).fit(X_train, np.log(y_train))

In [40]:
rand_predict = np.exp(rand_forest.predict(X_test))

Теперь вычислим наши метрики

In [41]:
books_forest_mape = (mape(y_test, rand_predict))*100
books_forest_mae = mae(y_test, rand_predict)
books_forest_rmse = math.sqrt(mean_squared_error(y_test, rand_predict))

In [42]:
print(f"TEST mape: {books_forest_mape:0.2f}%")

In [43]:
print(f"TEST mae: {books_forest_mae:0.2f}")

In [44]:
print(f"TEST rmse: {books_forest_rmse:0.2f}")

Значение МАРЕ еще немного уменьшилось, то есть наблюдается положительная динамика относительно используемых алгоритмов. " Грубые" ошибки и средняя абсолютная ошибка тоже немного уменьшились. Таким образом, мы получили более качественную модель по всем исследуемым метрикам. Далее перейдем к модели, использующей Catboost.

# Catboost

Catboost является комбинацией алгоритмов, использующих деревья и градиентный бустинг. Вызовем и обучим модель. Указано 60000 итераций, но фактически - будет выполняться пока не сработает прерывание.

In [45]:
model = CatBoostRegressor(iterations=60000,  # Количество итераций
                          learning_rate=0.1,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,  # Прерывает выполнение, если нет улучшения 500 итераций
                          # task_type='GPU',
                          )
# обучим модель
model.fit(X_train, np.log(y_train),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=100,
          use_best_model=True,
          # plot=True
          )

Спрогнозируем стоимость для валидационной выборки и вычислим метрики:

In [46]:
test_predict_catboost = np.exp(model.predict(X_test))

In [47]:
books_cat_mape = (mape(y_test, test_predict_catboost))*100
books_cat_mae = mae(y_test, test_predict_catboost)
books_cat_rmse = math.sqrt(mean_squared_error(y_test, test_predict_catboost))

In [48]:
print(f"TEST mape: {books_cat_mape:0.2f}%")

In [49]:
print(f"TEST mae: {books_cat_mae:0.2f}")

In [50]:
print(f"TEST rmse: {books_cat_rmse:0.2f}")

Сам по себе алгоритм без GPU вычисляет дольше предыдущих, но при этом имеет огромное преимущество в метриках: по сравнению с предыдущими результатами МАРЕ улучшилась больше чем в два раза, МАЕ и RMSE примерно в полтора раза. Таким образом, за несколько алгоритмов достигнута отметка, в 5~ раз лучше бейзлайна, который был в начале, что является хорошим результатом. Но впереди еще несколько нейронных сетей, и необходимо посмотреть, какие результаты покажут они.

# MLP сеть

Обычная нейронная сеть, собранная из нескольких полносвязных слоев, без добавления дропаутов. Структура сети выбрана на основе советов по предыдущим проектам, по этой же причине исключены dropout - слои. Создадим и скомпилируем модель:

In [51]:
model_ml = Sequential()
model_ml.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_ml.add(L.LayerNormalization())
model_ml.add(L.Dense(512, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-7), activation="relu"))
model_ml.add(L.Dense(1, activation="linear"))

In [52]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.001)
model_ml.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

Объявим простые коллбэки, такие как ранняя остановка, если нет положительной динамики на протяжении 50 эпох. Также используется чекпоинт для сохранения лучших весов модели

In [53]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [54]:
history = model_ml.fit(X_train, (y_train),
                       batch_size=512,  # размер батча
                       epochs=500,  # количество эпох для обучения
                       # данные для валидации
                       validation_data=(X_test, (y_test)),
                       callbacks=callbacks_list,  # список  наших коллбэков
                       verbose=1,  # параметр, отвечающий за выведение прогресс-бара
                       )

In [55]:
test_predict_nn1 = model_ml.predict(X_test)

Вычислим метрики:

In [56]:
books_mlp_mape = (mape(y_test, test_predict_nn1[:, 0]))*100
books_mlp_mae = mae(y_test, test_predict_nn1[:, 0])
books_mlp_rmse = math.sqrt(mean_squared_error(y_test, test_predict_nn1[:, 0]))

In [57]:
print(f"TEST mape: {books_mlp_mape:0.2f}%")

In [58]:
print(f"TEST mae: {books_mlp_mae:0.2f}")

In [59]:
print(f"TEST rmse: {books_mlp_rmse:0.2f}")

С MLP сетью ситуация получилась немного неоднозначная: Мы получили улучшение метрики МАРЕ, значит, в среднем, наша сеть стала меньше ошибаться при прогнозе. Но при этом метрики МАЕ и RMSE выросли, что значит, наша сеть делает более грубые ошибки, чем Catboost, который был рассмотрен до этого. Возникает извечный вопрос о балансе качества и количества. Общее направление у нас улучшилось, но темпы уже перестали быть такими высокими. Посмотрим, как поведет себя multi-input сеть, состоящая и MLP и NLP сетей.

# NLP+MLP сеть

## Предобработка текста

В данном дата-сете текст представлен в столбце "description". Этот столбец будет в дальнейшем использоваться для обучения NLP-сети. Для начала необходимо провести предварительную подготовку и создание функции для лемматизации и очистки текста.
Лемматизация - приведение слова к первоначальной словарной форме. Для этого используется pymorphy.
Под очисткой подразумевается удаление символов, не несущих смысловой нагрузки. 

In [99]:
morphy = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

In [97]:
# Паттерн с символами
trash_sym = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-–»«•∙·✔➥●☛“”°№₽®]+"

# функция для лемматизации текста:


def lemma(text):
    text = text.lower()  # понижаем регистр
    text = re.sub(trash_sym, ' ', text)  # удаляем символы из паттерна
    strings = []  # создаем массив, в котором будут храниться лемматизированные строки
    for wrd in text.split():  # берем слово из строки
        wrd = wrd.strip()  # убираем пробелы до и после слова
        wrd = morphy.normal_forms(wrd)[0]  # приводим к нормальной форме
        strings.append(wrd)  # добавляем слово в строку массива
    return ' '.join(strings)  # вернем значения, разделив пробелами

Сформируем список строк, применив к нашему столбцу функцию по лемматизации и очистке текста:

In [62]:
strings_set = []
strings_set = df_NLP.apply(
    lambda df_NLP: lemma(df_NLP.description), axis=1)

В любом языке существуют слова, не несущие смысловой нагрузки. Русский язык - не исключение. Поэтому уберем такие слова из наших строк при помощи stopwords:

In [104]:
russian_stopwords = stopwords.words("russian")

In [64]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]


# применим функцию к нашим лемматизированым строкам слов
str_without_stop = [lineWithoutStopWords(line) for line in strings_set]

In [65]:
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]

In [66]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [67]:
% % time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

In [68]:
% % time
text_train_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)

## NLP

In [69]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LayerNormalization())
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(128, activation="relu"))
model_nlp.add(L.LSTM(64,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

## MLP

In [70]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model_mlp.add(L.Dropout(0.25))

In [71]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(32, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [72]:
optimizer = tf.keras.optimizers.Adam(0.001)
# Проведем компиляцию модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [73]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=15, restore_best_weights=True,)
lr_scheduler = ReduceLROnPlateau(monitor='val_MAPE',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=10,  # если нет улучшения через 2 эпохи - уменьшить lr
                                 min_lr=0.00001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [74]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500,
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                    )

In [75]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])

In [76]:
books_multi_mape = (mape(y_test, test_predict_nn2[:, 0]))*100
books_multi_mae = mae(y_test, test_predict_nn2[:, 0])
books_multi_rmse = math.sqrt(
    mean_squared_error(y_test, test_predict_nn2[:, 0]))

In [77]:
print(f"TEST mape: {books_multi_mape:0.2f}%")

In [78]:
print(f"TEST mae: {books_multi_mae:0.2f}")

In [79]:
print(f"TEST rmse: {books_multi_rmse:0.2f}")

Как можно заметить все исследуемые метрики немного ухудшились, по сравнению с одиночной MLP-сетью. Вероятно выбрана не самая оптимальная структура обеих сетей, но тем не менее результат отличается незначительно, и он все еще гораздо лучше изначального бейзлайна. Теперь стоит перейти к нейронной сети TabNet.

# TabNet с K-Fold

Сеть TabNet на данный момент является SotA решением во множестве задач, она используется как для классификации, так и для задач регрессии. Но, как и у всех нейронных сетей, большинство проблем при обучении возникает из-за недостаточной полноты данных в выборке. 
Перейдем к обучению:

In [80]:
X_train_new = X_train.values
y_train_new = np.log(y_train).reshape(-1, 1)
X_test_new = X_test.values
y_test_new = np.log(y_test).reshape(-1, 1)

Так как у TabNet нет метрики МАРЕ, создадим ее сами для дальнейшего использования:

In [81]:
class my_Mape(Metric):
    def __init__(self):
        self._name = "mape"
        self._maximize = False

    def __call__(self, y_true, y_score):
        return np.mean(np.abs((y_score-y_true)/y_true))

Теперь приступим к разделению на обучающую и валидационную выборки с помощью KFold и самому обучению:

In [82]:
kf = KFold(n_splits=5, random_state=RANDOM_SEED, shuffle=True)
predictions_array = []
CV_score_array = []
for train_index, test_index in kf.split(X_train_new):
    X_train1, X_valid1 = X_train_new[train_index], X_train_new[test_index]
    y_train1, y_valid1 = y_train_new[train_index], y_train_new[test_index]
    regressor = TabNetRegressor(verbose=1, seed=RANDOM_SEED)
    regressor.fit(X_train=X_train1, y_train=y_train1,
                  eval_set=[(X_valid1, y_valid1)],
                  patience=1, max_epochs=5,
                  eval_metric=[my_Mape])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.exp(regressor.predict(X_test_new)))

predictions = np.mean(predictions_array, axis=0)

Искомые метрики:

In [ ]:
books_tnkf_mape = (mape(y_test, predictions))*100
books_tnkf_mae = mae(y_test, predictions)
books_tnkf_rmse = math.sqrt(mean_squared_error(y_test, predictions))

In [ ]:
print(f"TEST mape: {books_tnkf_mape:0.2f}%")

In [ ]:
print(f"TEST mae: {books_tnkf_mae:0.2f}")

In [ ]:
print(f"TEST rmse: {books_tnkf_rmse:0.2f}")

Как можно увидеть - все метрики ухудшились по сравнению с двумя предыдущими нейронными сетями. Сделать однозначный вывод, почему это произошло - довольно сложно. Но я перепробовал различные параметры при обучении, и улучшить показатели мне так и не удалось. Вероятно, это связано с качеством обучающих данных - мало столбцов, которые несут в себе нужные данные. Плюс сам дата сет не слишком длинный. На сколько я понял для TabNet рекомендуют использовать объемные дата-сеты. Посмотрим, как поведет себя TabNet без KFold.

# TabNet обычный

In [ ]:
regressor1 = TabNetRegressor(verbose=1, seed=RANDOM_SEED)

In [ ]:
regressor1.fit(X_train=X_train.values, y_train=np.log(y_train.reshape(-1, 1)),
               eval_set=[(X_test.values, np.log(y_test.reshape(-1, 1)))],
               patience=5, max_epochs=20,
               eval_metric=[my_Mape],
               batch_size=2048,
               virtual_batch_size=512)

In [ ]:
predict_stream = np.exp(regressor1.predict(X_test.values))

Вычислим метрики:

In [ ]:
books_tnst_mape = (mape(y_test, predict_stream))*100
books_tnst_mae = mae(y_test, predict_stream)
books_tnst_rmse = math.sqrt(mean_squared_error(y_test, predict_stream))

In [ ]:
print(f"TEST mape: {books_tnst_mape:0.2f}%")

In [ ]:
print(f"TEST mae: {books_tnst_mae:0.2f}")

In [ ]:
print(f"TEST rmse: {books_tnst_rmse:0.2f}")

Качество даже еще немного понизилось по сравнению с kfold. На данный момент табнет показывает результаты лучше чем линейная регрессия и случайный лес, но хуже чем все остальные.

# Выводы

По первому дата-сету мы получили примерно следующие результаты:

In [ ]:
data_metrics = [('baseline', np.round(books_base_mape), np.round(books_base_mae), np.round(books_base_rmse)),
                ('LinearRegression', np.round(books_linreg_mape), np.round(
                    books_linreg_mae), np.round(books_linreg_rmse)),
                ('RandomForest', np.round(books_forest_mape), np.round(
                    books_forest_mae), np.round(books_forest_rmse)),
                ('CatBoost', np.round(books_cat_mape), np.round(
                    books_cat_mae), np.round(books_cat_rmse)),
                ('MLP', np.round(books_mlp_mape), np.round(
                    books_mlp_mae), np.round(books_mlp_rmse)),
                ('Multi-Input', np.round(books_multi_mape),
                 np.round(books_multi_mae), np.round(books_multi_rmse)),
                ('TabNet KFold', np.round(books_tnkf_mape), np.round(
                    books_tnkf_mae), np.round(books_tnkf_rmse)),
                ('TabNet Stream', np.round(books_tnst_mape), np.round(
                    books_tnst_mae), np.round(books_tnst_rmse))
                ]
labels = ['model', 'MAPE', 'MAE', 'RMSE']
df = pd.DataFrame.from_records(data_metrics, columns=labels)

In [ ]:
df

По данной таблице можно сказать, что первое место по всем параметрам занимает обычная MLP сеть, состоящая из нескольких Dense-слоев, а следующая после нее идет multi-input сеть. Метрики двух этих сетей являются наиболее низкими, что означает уменьшение количества и грубости ошибок по сравнению с другими моделями. 

# Второй дата-сет (целевой)

Дата-сет с играми содержит данные о настольных играх, собранные с помощью парсинга с сайта hobby-games.ru. Изначально, с ним возникли трудности из-за малого количества данных для обучения, поэтому было произведено исследование на первом дата-сете, о том как повлияет увеличение выборки на исследуемые метрики.

Скопируем данные и уберем те, в которых не указана цена:

In [18]:
data_games1 = data_games.copy()

In [19]:
data_games1 = data_games1[data_games1.price != 'Цена скоро будет']

In [20]:
data_games1 = data_games1.reset_index(drop=True)

In [21]:
data_games1.head()

Представленные столбцы:
* gameName - название игры
* players - количество игроков
* time - среднее время одной партии
* age - возраст
* price - цена игры ( наш таргет)
* description - описание игры
* image - ссылка на изображение (Сильно усложняет обработку и не оказывает должного эффекта. Не используется)
* manufacturer - производитель
* package - Что входит в набор

# Небольшая предобработка

Уберем ненужную информацию из столбцов:

In [22]:
data_games1['players'] = data_games1['players'].apply(
    lambda x: x.split(' ')[0])
data_games1['time'] = data_games1['time'].apply(lambda x: x.split(' ')[0])

Очистим столбец с ценами и переведем в int:

In [23]:
data_games1['price'] = data_games1['price'].apply(
    lambda x: x.replace('бон.', ''))
data_games1['price'] = data_games1['price'].apply(lambda x: x.replace('₽', ''))
data_games1['price'] = data_games1['price'].apply(
    lambda x: x.replace('\xa0', ''))

In [24]:
data_games1['price'] = data_games1['price'].apply(lambda x: int(x))

Уберем некоторые ненужные символы из описания и состава игры для выделения некоторых фич:

In [25]:
data_games1['description'] = data_games1['description'].apply(
    lambda x: x.replace('\r', ''))
data_games1['description'] = data_games1['description'].apply(
    lambda x: x.replace('\n', ''))
data_games1['description'] = data_games1['description'].apply(
    lambda x: x.replace('\t', ''))
data_games1['package'] = data_games1['package'].apply(
    lambda x: x.replace('\r', ''))
data_games1['package'] = data_games1['package'].apply(
    lambda x: x.replace('\n', ''))
data_games1['package'] = data_games1['package'].apply(
    lambda x: x.replace('\t', ''))

# Feature engeneering

На данном этапе выделим несколько фич для обучения:
* Есть ли карты в наборе
* Есть ли фигурки в наборе
* Символьная длина описания
* Символьная длина набора
* Символьная длина названия

In [26]:
data_games1['cards'] = data_games1['package'].apply(
    lambda x: 1 if 'карт' in x else 0)
data_games1['figures'] = data_games1['package'].apply(
    lambda x: 1 if 'фигур' in x else 0)
data_games1['pack_length'] = data_games1['package'].apply(lambda x: len(x))
data_games1['desc_length'] = data_games1['description'].apply(lambda x: len(x))
data_games1['name_length'] = data_games1['gameName'].apply(lambda x: len(x))

In [27]:
data_games1.head()

Теперь построим гистограммы для наших новых столбцов:

In [139]:
data_games1.cards.hist()

In [140]:
data_games1.figures.hist()

Как можно увидеть - данные несбалансированы. Скорее всего это связано с тем, что большинство настольных игр делают довольно простыми, в них присутствуют только карты из дополнительных вещей игры. Фигурки - вещь гораздо менее используемая, однако наличие фигурок, по собственному опыту могу сказать, обычно сильно увеличивает стоимость самой игры.

Категориальные и числовые столбцы, используемые для обучения:

In [28]:
cat_features1 = ['players', 'time', 'age', 'manufacturer', 'cards', 'figures']
num_features1 = ['pack_length', 'desc_length', 'name_length']

Функция для предобработки данных:

In [29]:
def preproc_data1(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'package', 'gameName',
                    'image'], axis=1, inplace=True)

    scaler = RobustScaler()
    for column in num_features1:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features1:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features1, dummy_na=False)

    return df_output

In [30]:
df_preproc1 = preproc_data1(data_games1)
df_preproc1.head(10)

Разделение на выборки:

In [31]:
y1 = df_preproc1.price.values     # наш таргет
X1 = df_preproc1.drop(['price'], axis=1)

In [32]:
X_train_games, X_test_games, y_train_games, y_test_games = train_test_split(
    X1, y1, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

# Бейзлайн

Бейзлайн необходим для того, чтобы иметь понимание в какую сторону мы движемся при использовании различных моделей прогнозирования стоимости.

In [34]:
game_base_mape = (mape(y_test_games, np.mean(y_train_games)))*100
game_base_mae = mae(y_test_games, np.mean(y_train_games))
game_base_rmse = rmse(y_test_games, np.mean(y_train_games))

In [44]:
print(f"TEST mape: {game_base_mape:0.2f}%")
print(f"TEST mae: {game_base_mae:0.2f}")
print(f"TEST rmse: {game_base_rmse:0.2f}")

Как можно увидеть, согласно метрике МАРЕ ,в среднем, такая модель ошибается более чем на 100% при определении стоимости игры. Данный результат является очень плохим, фактически неспособным дать хоть какой-то достоверный прогноз. Но он нужен для определении базы для дальнейшего выбора наиболее продуктивного алгоритма.

# Линейная регрессия

Как и в дата-сете с книгами, первым делом посмотрим, как будет себя вести линейная регрессия, какие метрики она покажет:

In [37]:
linreg = LinearRegression().fit(X_train_games, np.log(y_train_games))
predict_linreg = np.exp(linreg.predict(X_test_games))

Возникла та же самая проблема, некоторые значения при экспоненцировании уходят в inf. Избавимся от них для вычисления метрик

In [46]:
y_test_new1 = []
linreg_pred = []

In [47]:
for i in range(0, len(y_test_games)-1):
    if predict_linreg[i] != np.inf:
        y_test_new1.append(y_test_games[i])
        linreg_pred.append(predict_linreg[i])

In [48]:
y_test_new1 = np.array(y_test_new1)
linreg_pred = np.array(linreg_pred)

In [49]:
game_linreg_mape = (mape(y_test_new1, linreg_pred))*100
game_linreg_mae = mae(y_test_new1, linreg_pred)
game_linreg_rmse = rmse(y_test_new1, linreg_pred)

In [50]:
print(f"TEST mape: {game_linreg_mape:0.2f}%")
print(f"TEST mae: {game_linreg_mae:0.2f}")
print(f"TEST rmse: {game_linreg_rmse:0.2f}")

Погрешность упала, по сравнению с бейзлайном. Также, можно выделить, что погрешность почти в два раза отличается от дата-сета с книгами, выходит что на текущих данных регрессия чувствует себя лучше, чем на тех, которые использовались в начале. Посмотрим, как поведут себя другие алгоритмы.

# RandomForest

RandomForest используется с теми же параметрами, которые были в первой части работы. Необходимо отдельно отметить, что были попытки использовать различные параметры в поисках улучшения метрик, однако это не привело к разительным успехам. Кроме того, действительно хорошим решением было бы использовать подбор гиперпараметров, однако этот способ занимает много вычислительного времени.

In [52]:
game_forest = RandomForestRegressor(n_estimators=500,
                                    n_jobs=-1,
                                    max_depth=15,
                                    max_features='log2',
                                    random_state=RANDOM_SEED,
                                    oob_score=True).fit(X_train_games, np.log(y_train_games))

rf_predict_games = np.exp(game_forest.predict(X_test_games))

In [53]:
game_forest_mape = (mape(y_test_games, rf_predict_games))*100
game_forest_mae = mae(y_test_games, rf_predict_games)
game_forest_rmse = rmse(y_test_games, rf_predict_games)

In [54]:
print(f"TEST mape: {game_forest_mape:0.2f}%")
print(f"TEST mae: {game_forest_mae:0.2f}")
print(f"TEST rmse: {game_forest_rmse:0.2f}")

Первое отличие от исследования на дата-сете с книгами - там случайный лес показал улучшение метрик, по сравнению с линейной регрессией. Здесь же получилось, что его метрики немного хуже, вероятно для него недостаточно данных, чтобы найти наиболее качественное решение задачи. Посмотрим, как это скажется на других моделях.

# Catboost

In [55]:
cb_model = CatBoostRegressor(iterations=60000,  # Количество итераций
                             learning_rate=0.1,
                             random_seed=RANDOM_SEED,
                             eval_metric='MAPE',
                             custom_metric=['RMSE', 'MAE'],
                             od_wait=500,  # Прерывает выполнение, если нет улучшения 500 итераций
                             )
# обучим модель
cb_model.fit(X_train_games, np.log(y_train_games),
             eval_set=(X_test_games, np.log(y_test_games)),
             verbose_eval=100,
             use_best_model=True,
             )

In [56]:
game_predict_catboost = np.exp(cb_model.predict(X_test_games))

In [58]:
game_cat_mape = (mape(y_test_games, game_predict_catboost))*100
game_cat_mae = mae(y_test_games, game_predict_catboost)
game_cat_rmse = rmse(y_test_games, game_predict_catboost)

In [59]:
print(f"TEST mape: {game_cat_mape:0.2f}%")
print(f"TEST mae: {game_cat_mae:0.2f}")
print(f"TEST rmse: {game_cat_rmse:0.2f}")

Catboost все так же показывает хорошие результаты, он неизменимо улучшает исследуемые метрики. Так, при catboost'e, наши прогнозы, в среднем, на 41% отличаются от фактических значений выборки, что гораздо лучше, чем 117% при бейзлайне.

# MLP сеть

Данная модель идентична с моделью из первой части работы и собрана из нескольких Dense-слоев с использованием l2 регуляризации и функции активации нейрона "relu". Создадим и скомпилируем модель:

In [82]:
model_ml1 = Sequential()
model_ml1.add(
    L.Dense(1024, input_dim=X_train_games.shape[1], activation="relu"))
model_ml1.add(L.Dense(512, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml1.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml1.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-7), activation="relu"))
model_ml1.add(L.Dense(1, activation="linear"))

In [83]:
# Compile model
optimizer = tf.keras.optimizers.Adamax(0.001)
model_ml1.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

Пропишемо основные колбэки:

In [84]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

Обучение:

In [85]:
history1 = model_ml1.fit(X_train_games, y_train_games,
                         batch_size=512,  # размер батча
                         epochs=500,  # количество эпох для обучения
                         # данные для валидации
                         validation_data=(X_test_games, y_test_games),
                         callbacks=callbacks_list,  # список  наших коллбэков
                         verbose=1,  # параметр, отвечающий за выведение прогресс-бара
                         )

In [90]:
game_predict_nn1 = model_ml1.predict(X_test_games)

In [91]:
game_mlp_mape = (mape(y_test_games, game_predict_nn1[:, 0]))*100
game_mlp_mae = mae(y_test_games, game_predict_nn1[:, 0])
game_mlp_rmse = rmse(y_test_games, game_predict_nn1[:, 0])

In [92]:
print(f"TEST mape: {game_mlp_mape:0.2f}%")
print(f"TEST mae: {game_mlp_mae:0.2f}")
print(f"TEST rmse: {game_mlp_rmse:0.2f}")

Как можно увидеть - все метрика МАРЕ еще немного улучшилась, однако метрики МАЕ и RMSE стали немного хуже. Это значит, что наша модель, в среднем, ошибается на 37% в сравнении с фактическим значением, но грубые ошибки, которые могут возникать при прогнозе, стали немного больше. Необходимо отдельно выделить, что при попытке логарифмировать таргет модель давала характеристики, на 5-7% худшие, чем без него. 

# MLP+NLP+NLP

Здесь структура сети немного отличается от первой части, так как в NLP мы используем две сети, одна из которых обучается на столбце "description", а вторая на столбце "package". В остальном, процесс обработки данных такой же, как и в multi-input сети для дата-сета с книгами.

Обработка текста

In [93]:
df_NLP1 = data_games1.copy()

In [100]:
strings_set1 = []
strings_set1 = df_NLP1.apply(
    lambda df_NLP1: lemma(df_NLP1.description), axis=1)

In [101]:
strings_set2 = []
strings_set2 = df_NLP1.apply(
    lambda df_NLP1: lemma(df_NLP1.package), axis=1)

In [102]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]

In [105]:
# применим функцию к нашим лемматизированым строкам слов
str_without_stop1 = [lineWithoutStopWords(line) for line in strings_set1]
str_without_stop2 = [lineWithoutStopWords(line) for line in strings_set2]

In [106]:
text_train_games = data_games1.package.iloc[X_train_games.index]
text_test_games = data_games1.package.iloc[X_test_games.index]

In [107]:
text_train_games1 = data_games1.description.iloc[X_train_games.index]
text_test_games1 = data_games1.description.iloc[X_test_games.index]

In [108]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [109]:
% % time
tokenize1 = Tokenizer(num_words=MAX_WORDS)
tokenize1.fit_on_texts(str_without_stop1)

In [110]:
% % time
tokenize2 = Tokenizer(num_words=MAX_WORDS)
tokenize2.fit_on_texts(str_without_stop2)

In [111]:
% % time
text_train_sequences1 = sequence.pad_sequences(
    tokenize1.texts_to_sequences(text_train_games), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences1 = sequence.pad_sequences(
    tokenize1.texts_to_sequences(text_test_games), maxlen=MAX_SEQUENCE_LENGTH)

In [112]:
% % time
text_train_sequences2 = sequence.pad_sequences(
    tokenize2.texts_to_sequences(text_train_games1), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences2 = sequence.pad_sequences(
    tokenize2.texts_to_sequences(text_test_games1), maxlen=MAX_SEQUENCE_LENGTH)

# NLP1 сеть

In [113]:
model_nlp1 = Sequential()
model_nlp1.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package1"))
model_nlp1.add(L.Embedding(len(tokenize1.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp1.add(L.LayerNormalization())
model_nlp1.add(L.LSTM(256, return_sequences=True))
model_nlp1.add(L.Dropout(0.5))
model_nlp1.add(L.Dense(128, activation="relu"))
model_nlp1.add(L.LSTM(64,))
model_nlp1.add(L.Dropout(0.25))
model_nlp1.add(L.Dense(64, activation="relu"))

# NLP2 сеть

In [114]:
model_nlp2 = Sequential()
model_nlp2.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package2"))
model_nlp2.add(L.Embedding(len(tokenize2.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp2.add(L.LayerNormalization())
model_nlp2.add(L.LSTM(256, return_sequences=True))
model_nlp2.add(L.Dropout(0.5))
model_nlp2.add(L.Dense(128, activation="relu"))
model_nlp2.add(L.LSTM(64,))
model_nlp2.add(L.Dropout(0.25))
model_nlp2.add(L.Dense(64, activation="relu"))
model_nlp2.add(L.Dropout(0.25))

# MLP сеть

In [115]:
model_mlp1 = Sequential()
model_mlp1.add(
    L.Dense(512, input_dim=X_train_games.shape[1], activation="relu"))
model_mlp1.add(L.Dropout(0.5))
model_mlp1.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_mlp1.add(L.Dropout(0.5))
model_mlp1.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model_mlp1.add(L.Dropout(0.25))

In [116]:
combinedInput = L.concatenate(
    [model_nlp1.output, model_nlp2.output, model_mlp1.output])
# being our regression head
head = L.Dense(32, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp1.input, model_nlp2.input,
                      model_mlp1.input], outputs=head)

In [117]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=30, restore_best_weights=True,)
lr_scheduler = ReduceLROnPlateau(monitor='val_MAPE',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=10,  # если нет улучшения через 2 эпохи - уменьшить lr
                                 min_lr=0.00001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [118]:
optimizer = tf.keras.optimizers.Adam(0.001)
# Проведем компиляцию модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [119]:
history = model.fit([text_train_sequences1, text_train_sequences2, X_train_games], y_train_games,
                    batch_size=512,
                    epochs=500,
                    validation_data=(
                        [text_test_sequences1, text_test_sequences2, X_test_games], y_test_games),
                    callbacks=callbacks_list
                    )

In [120]:
game_predict_nn2 = model.predict(
    [text_test_sequences1, text_test_sequences2, X_test_games])

In [121]:
game_multi_mape = (mape(y_test_games, game_predict_nn2[:, 0]))*100
game_multi_mae = mae(y_test_games, game_predict_nn2[:, 0])
game_multi_rmse = rmse(y_test_games, game_predict_nn2[:, 0])

In [122]:
print(f"TEST mape: {game_multi_mape:0.2f}%")
print(f"TEST mae: {game_multi_mae:0.2f}")
print(f"TEST rmse: {game_multi_rmse:0.2f}")

Как видно, метрики остались примерно на том же самом уровне, что и при одиночной MLP сети. То есть, надстройка из еще двух сетей для обработки текста не оказала негативного эффекта, как в первой части работы. Поэтому на данный момент, мне кажется, лидирующее место можно отдать именно этой конструкции.

# Простой TabNet

Табнет с kfold разбиением я решил не рассматривать, так как метрики у них получаются примерно одинаковые, однако придется создавать дополнительные переменные для разбивки и обучения. Поэтому оставлен только TabNet обычный.

In [127]:
regressor2 = TabNetRegressor(verbose=1, seed=RANDOM_SEED)

In [128]:
regressor2.fit(X_train=X_train_games.values, y_train=np.log(y_train_games.reshape(-1, 1)),
               eval_set=[(X_test_games.values, np.log(
                   y_test_games.reshape(-1, 1)))],
               patience=5, max_epochs=20,
               eval_metric=['mae', 'mse', 'rmse'],
               batch_size=2048,
               virtual_batch_size=512)

In [129]:
predict_stream1 = np.exp(regressor2.predict(X_test_games.values))

In [131]:
game_tabnet_mape = (mape(y_test_games, predict_stream1))*100
game_tabnet_mae = mae(y_test_games, predict_stream1)
game_tabnet_rmse = rmse(y_test_games, predict_stream1)

In [132]:
print(f"TEST mape: {game_tabnet_mape:0.2f}%")
print(f"TEST mae: {game_tabnet_mae:0.2f}")
print(f"TEST rmse: {game_tabnet_rmse:0.2f}")

TabNet, как и в первой части работы, оказался неэффективен для этих данных. Скорее всего, для него нужно гораздо больше параметров, которые он будет использовать при обучении. Но данные, собранные в процессе парсинга, к сожалению, не могут предоставить столько ресурсов для обучения.

# Выводы

Рассмотрим результаты метрик, которые получены при реализации различных моделей, направленных на предсказание стоимости настольных игр:

In [141]:
data_metrics1 = [('baseline', np.round(game_base_mape), np.round(game_base_mae), np.round(game_base_rmse)),
                 ('LinearRegression', np.round(game_linreg_mape),
                  np.round(game_linreg_mae), np.round(game_linreg_rmse)),
                 ('RandomForest', np.round(game_forest_mape), np.round(
                     game_forest_mae), np.round(game_forest_rmse)),
                 ('CatBoost', np.round(game_cat_mape), np.round(
                     game_cat_mae), np.round(game_cat_rmse)),
                 ('MLP', np.round(game_mlp_mape), np.round(
                     game_mlp_mae), np.round(game_mlp_rmse)),
                 ('Multi-Input', np.round(game_multi_mape),
                  np.round(game_multi_mae), np.round(game_multi_rmse)),
                 ('TabNet', np.round(game_tabnet_mape), np.round(
                     game_tabnet_mae), np.round(game_tabnet_rmse))
                 ]
labels = ['model', 'MAPE', 'MAE', 'RMSE']
df1 = pd.DataFrame.from_records(data_metrics1, columns=labels)

In [142]:
df1

В общем случае получился следующий результат: Одиночная MLP и Multi-input сети имеют наилучшие показатели метрики МАРЕ среди всех рассматриваемых моделей. Если же рассматривать общие показатели всех трех метрик, то им конкуренцию составляет модель CatBoost: она имеет чуть более высокую среднюю погрешность, чем нейросети, но при этом показатели МАЕ и RMSE у него меньше, что означает об уменьшении среднего значения грубых ошибок. 

Далее, основываясь на собвстенном опыте, хотел бы выделить, что сейчас стоимость настольных игр постоянно меняется, поэтому руководствоваться только таким маленьким дата-сетом не очень правильно, лучше выделить и исследовать тенденцию с течением времени ( нескольких месяцев), чтобы иметь дата-сеты разреженные по времени. Так, скорее всего, получится достигнуть более стабильных показателей. У настольных игр, к сожалению, не так много первичных параметров, которые можно было бы выделить для большей информативности. В дополнение - не все магазины, с сайтов которых парсится информация, удобоваримо выделяют различные параметры игры в нормальные классы, что позволяет проводить парсинг спокойно, без каких либо эксцессов.

# Итог

Подводя итог, хотел бы сказать отдельно, что парсинг данных был очень сложным, не в последнюю очередь из-за постоянно меняющихся цен. Это, скорее всего оказало негативное влияние на имеющиеся данные. Сами данные были очищены и был проведен небольшой EDA и feature-engeneering. Были исследованы различные алгоритмы на двух наборах данных, но общие тенденцие наблюдаются примерно схожие.

В качестве модели, которая будет решать поставленную задачу, на мой взгляд, стоит выбрать Multi-input сеть, так как ее метрики не сильно различаются с лидирующей "MLP", однако она имеет небольшое преимущество, так как ее можно будет дополнять различными надстройками, способными и дальше улучшать качество модели посредством уменьшения погрешностей.